In [1]:
import os
print(os.getcwd())

import scipy
import numpy as np
import tensorflow as tf

/home/marc/projects/metamotif


2022-06-07 20:54:18.863182: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 20:54:18.863232: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
base2int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def sequence2int(sequence):
    return [base2int.get(base, 999) for base in sequence]

def sequence2onehot(sequence):
    return tf.one_hot(sequence2int(sequence), depth=4)

In [3]:
def load_kmers(tsv):
    kmers_onehot = []
    with open(tsv) as f:
        _ = f.readline()
        for line in f:
            name, kmer, score = line.strip().split('\t')
            kmers_onehot.append(sequence2onehot(kmer))
    return np.stack(kmers_onehot)

In [4]:
kmers_onehot = load_kmers('examples/RBFOX2_HepG2.5mers.tsv')

2022-06-07 20:54:20.358396: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-07 20:54:20.359258: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-07 20:54:20.391573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 20:54:20.391813: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3c:00.0 name: NVIDIA GeForce MX250 computeCapability: 6.1
coreClock: 1.582GHz coreCount: 3 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 52.21GiB/s
2022-06-07 20:54:20.391888: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or

In [5]:
pwm = kmers_onehot[1]
pwm

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [6]:
from metamotif.similarity import motif_similarity

In [7]:
%%timeit
motif_similarity(pwm, pwm, min_size=3)

2022-06-07 20:54:28.527359: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-07 20:54:28.547162: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz


5.59 ms ± 478 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
def motif_similarity_to_reference(pwm, reference, reduce=tf.reduce_max, **kwargs):
    """Compute the similarity between a PWM and a (list) of reference PWMs. 
    
    Similarity between the PWM and a single reference is computed via motif_similarity, 
    using provided kwargs. Similarity values between the PWM and all references are reduce via a 
    reduction function (default: max). 

    Args:
        pwm (tf.Tensor): PWM. 
        reference (tf.Tensor or list): (List of) reference PWMs.
        reduce (function, optional): Reduce function. Defaults to tf.reduce_max.

    Returns:
        tf.Tensor: Scalar of similarity to references. 
    """
    
    if isinstance(reference, list):
        pass
    else:
        reference = [reference]
    return reduce([motif_similarity(pwm, pwm_r, **kwargs) for pwm_r in reference])

In [9]:
motif_similarity_to_reference(pwm, [pwm, pwm, pwm])

<tf.Tensor: shape=(), dtype=float64, numpy=1.0>

In [10]:
pwm_fn = '/home/marc/Downloads/RBP_PSSMs/CNOT4_gacaga_human_PSSM.txt'

In [11]:
def load_pwm(fname):
    with open(fname) as f:
        pwm = list()
        header = f.readline()
        if header[:2] == 'ID':
            _ = f.readline()
        for line in f:
            row = line.strip().split('\t')[1:]
            if len(row) != 4:
                break
            pwm.append(list(map(float, row)))
        pwm = np.array(pwm)
    return pwm

pwm = load_pwm(pwm_fn)
print(pwm)

[[0.02892607 0.16888298 0.77326488 0.02892607]
 [0.98456113 0.00514629 0.00514629 0.00514629]
 [0.00514629 0.91451045 0.07519696 0.00514629]
 [0.98456113 0.00514629 0.00514629 0.00514629]
 [0.00514629 0.1841677  0.80553971 0.00514629]
 [0.98456113 0.00514629 0.00514629 0.00514629]
 [0.30100821 0.18412659 0.20970436 0.30516084]]


In [12]:
pwm_dir = '/home/marc/Downloads/RBP_PSSMs/'

In [13]:
from pathlib import Path

def load_pwms(pwm_dir):
    pwms = dict()
    for pwm_txt in Path(pwm_dir).glob('*_human_PSSM.txt'):
        RBP = pwm_txt.name.split('_')[0]
        if RBP not in pwms:
            pwms[RBP] = list()
        pwms[RBP].append(load_pwm(pwm_txt))
    return pwms

pwms = load_pwms(pwm_dir)

In [14]:
pwms['QKI']

[array([[1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.57, 0.  , 0.43],
        [0.  , 0.  , 0.  , 1.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.83, 0.  , 0.17]])]

In [21]:
def load_kmers(tsv):
    kmers_onehot = []
    with open(tsv) as f:
        _ = f.readline()
        for line in f:
            name, kmer, score = line.strip().split('\t')
            kmers_onehot.append(sequence2onehot(kmer))
    return np.stack(kmers_onehot)

QKI_5mers_target = load_kmers('/home/marc/Downloads/5mers_processed/QKI_HepG2/5mer.RBFOX2_HepG2.profile_target.csv')
QKI_5mers_control = load_kmers('/home/marc/Downloads/5mers_processed/QKI_HepG2/5mer.QKI_HepG2.profile_control.csv')
QKI_5mers_total = load_kmers('/home/marc/Downloads/5mers_processed/QKI_HepG2/5mer.QKI_HepG2.profile.csv')

In [35]:
motif_similarity_to_reference(QKI_5mers_target[0], pwms['QKI'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.9302546937479207>

In [30]:
QKI_5mers_target_sims = [motif_similarity_to_reference(mer, pwms['QKI'], min_size=3).numpy() for mer in QKI_5mers_target]
QKI_5mers_control_sims = [motif_similarity_to_reference(mer, pwms['QKI'], min_size=3).numpy() for mer in QKI_5mers_control]
QKI_5mers_total_sims = [motif_similarity_to_reference(mer, pwms['QKI'], min_size=3).numpy() for mer in QKI_5mers_total]

In [32]:
print(np.mean(QKI_5mers_target_sims))
print(np.mean(QKI_5mers_control_sims))
print(np.mean(QKI_5mers_total_sims))

0.9235518389296927
0.6644967164812121
0.9236690094906977


In [36]:
RBFOX2_5mers_target = load_kmers('/home/marc/Downloads/5mers_processed/RBFOX2_HepG2/5mer.RBFOX2_HepG2.profile_target.csv')
RBFOX2_5mers_control = load_kmers('/home/marc/Downloads/5mers_processed/RBFOX2_HepG2/5mer.RBFOX2_HepG2.profile_control.csv')
RBFOX2_5mers_total = load_kmers('/home/marc/Downloads/5mers_processed/RBFOX2_HepG2/5mer.RBFOX2_HepG2.profile.csv')

RBFOX2_5mers_target_sims = [motif_similarity_to_reference(mer, pwms['RBFOX2'], min_size=3).numpy() for mer in RBFOX2_5mers_target]
RBFOX2_5mers_control_sims = [motif_similarity_to_reference(mer, pwms['RBFOX2'], min_size=3).numpy() for mer in RBFOX2_5mers_control]
RBFOX2_5mers_total_sims = [motif_similarity_to_reference(mer, pwms['RBFOX2'], min_size=3).numpy() for mer in RBFOX2_5mers_total]

print(np.mean(RBFOX2_5mers_target_sims))
print(np.mean(RBFOX2_5mers_control_sims))
print(np.mean(RBFOX2_5mers_total_sims))

0.8159882319083178
0.6188980017111788
0.6628939076290632


In [22]:
sum(QKI_5mers_target_sims)/len(QKI_5mers_target_sims)

0.9235518389296756

In [ ]:
pwms

In [ ]:
print(pwm[0:3])
print(pwm[3:6])

In [ ]:
from metamotif.similarity import motif_point_similarity

motif_point_similarity(pwm[0:3], pwm[3:6])

In [ ]:
from metamotif.similarity import motif_similarity

motif_similarity(pwm[2:5], pwm[3:6], min_size=2)

In [ ]:
kmers_onehot.shape

In [ ]:
pwm = kmers_onehot[1]
pwm

In [ ]:
pwm_pad = tf.pad(pwm, [[2, 2,], [0, 0]], 'CONSTANT')
pwm_pad

In [ ]:
print(pwm_pad[0:5])
print(tf.reduce_sum(pwm_pad[0:5], axis=1))

In [ ]:
tf.slice(pwm_pad, [0, 0], [5, 4])

# tf.slice(pwm_pad, tf.range(pwm_pad.shape[0] - 5 + 1), [5, pwm_pad.shape[1]])

In [ ]:
tf.tile(pwm_pad, [1, 2])

In [ ]:
tf.strided_slice(pwm_pad, [0, 0], [5, 4], strides=[2, 2])

In [ ]:
pwm

In [ ]:
pwm_pad_windows = tf.squeeze(np.lib.stride_tricks.sliding_window_view(pwm_pad, window_shape=(5, 4), axis=None))
print(pwm_pad_windows.shape)
print(pwm_pad_windows)

In [ ]:
pwm_tiled = tf.reshape(tf.tile(pwm, [5, 1]), (5, 5, 4))
print(pwm_tiled.shape)
print(pwm_tiled)

In [ ]:
kld = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.NONE)

In [ ]:
kld(pwm_tiled, pwm_tiled)

In [ ]:
mask = tf.cast(tf.logical_and(tf.cast(np.tril(np.ones((5, 5)), k=2), tf.bool), tf.cast(np.tril(np.ones((5, 5)), k=2).transpose(), tf.bool)), tf.float32)
mask = mask[::-1]
mask

In [ ]:
kld = tf.keras.losses.KLDivergence()

In [ ]:
c = tf.stack([pwm_tiled, pwm_tiled], axis=1)

In [ ]:
a = tf.random.uniform(shape=(5, 4))
a

In [ ]:
tf.boolean_mask(a, tf.cast([0, 0, 1, 1, 0], tf.bool))

In [ ]:
@tf.function
def log(x, basis=None):
    if basis is None:
        return tf.math.log(x)
    else:
        return tf.math.log(x) / tf.math.log(tf.cast(basis, x.dtype))

@tf.function
def kld(q, p, basis=None):
    p = tf.convert_to_tensor(p)
    q = tf.cast(q, p.dtype)
    q = tf.keras.backend.clip(q, tf.keras.backend.epsilon(), 1)
    p = tf.keras.backend.clip(p, tf.keras.backend.epsilon(), 1)
    return tf.reduce_sum(q * log(q / p, basis=basis), axis=-1)

@tf.function
def jsd(p, q, logits=True):
    m = (p + q) / 2
    return 1 - (kld(p, m, basis=2)/2 + kld(q, m, basis=2)/2)

In [ ]:
@tf.function()
def motif_point_similarity(pwm_1, pwm_2, boolean_mask=None, weights=None, sim_fn=jsd):
    tf.debugging.assert_equal(pwm_1.shape, pwm_1.shape)
    
    sim = sim_fn(pwm_1, pwm_2)
    if weights is not None:
        sim = tf.multiply(sim, weights)
    if boolean_mask is not None:
        sim = tf.boolean_mask(sim, tf.cast(boolean_mask, tf.bool))
    return tf.reduce_mean(sim)


@tf.function()
def tile_pwm(pwm, n):
    #tf.debugging.assert_rank(pwm, 2)
    return tf.reshape(tf.tile(pwm, tf.constant([n, 1], dtype=tf.int64)), (n, pwm.shape[0], pwm.shape[1]))

def sliding_window_view(x, window_size):
    windows = np.zeros(shape = (x.shape[0] - window_size + 1, window_size, *x.shape[1:]))
    for i in range(windows.shape[0]):
        windows[i, ] = x[i:(i+window_size), ]
    return windows

@tf.function
def pwm_padded_windows(pwm, padding=0):
    pwm_pad = tf.pad(pwm, [[padding, padding,], [0, 0]], 'CONSTANT')
    return tf.py_function(func=sliding_window_view, inp=[pwm_pad, pwm.shape[0]], Tout=tf.float32)
    #return sliding_window_view(pwm_pad, window_size=5)
    #return tf.squeeze(np_sliding_window_view(pwm_pad, window_shape=(5, 4), axis=None))


@tf.function()
def _make_mask(pwm_1, pwm_2):
    return tf.cast(tf.logical_and(tf.cast(tf.reduce_sum(pwm_1, axis=1), tf.bool), tf.cast(tf.reduce_sum(pwm_2, axis=1), tf.bool)), tf.float32)

@tf.function()
def _map_masked_point_similarity(pwm_a_b):
    pwm_a, pwm_b = pwm_a_b[0], pwm_a_b[1]
    mask = _make_mask(pwm_a, pwm_b)
    return motif_point_similarity(pwm_a, pwm_b, boolean_mask=mask)

#@tf.function()
def motif_similarity(pwm_1, pwm_2, min_size=3, reduce=tf.reduce_max):
    # assign larger PWM to pwm_1
    if pwm_1.shape[0] < pwm_2.shape[0]:
        pwm_1, pwm_2 = pwm_2, pwm_1
    
    # pad the longer PWM and create sliding windows over it
    pwm_1_padded_windows = pwm_padded_windows(pwm_1, padding=(pwm_2.shape[0] - min_size))
    #print(pwm_1_padded_windows[0])
    
    # tile the shorted PWM to match the number of sliding windows
    pwm_2_tiled = tile_pwm(pwm_2, pwm_1_padded_windows.shape[0])
    #print(pwm_2_tiled[0])

    # compute the dinstance between pwm_2 and all windows of pwm_1
    window_sims = tf.map_fn(_map_masked_point_similarity, tf.stack([pwm_1_padded_windows, pwm_2_tiled], axis=1))

    return window_sims
    return reduce(window_sims)

#print(_tile_pwm(pwm, n=3))
#print(_pwm_padded_windows(pwm, 2))
print(motif_similarity(pwm, pwm, min_size=3))

print(motif_point_similarity(pwm_pad_windows[0], pwm, boolean_mask=[0,0,1,1,1]))
# @tf.function()
# def motif_sim(pwm_a, pwm_b, position_weights=None, sim=kld):
#     return tf.map_fn(_map_sim, [pwm_a, pwm_b], dtype=tf.float32)

# motif_point_similarity(pwm, pwm)

# tf.map_fn(_map_masked_point_similarity, tf.stack([pwm_tiled, pwm_tiled], axis=1))

In [ ]:
pwm_pad_windows

In [ ]:
motif_point_similarity(pwm_pad_windows[4], pwm)

In [ ]:
jsd(pwm_pad_windows[4], pwm)

In [ ]:
def sliding_window_view(x, window_size):
    windows = np.zeros(shape = (len(x) - window_size + 1, window_size, *x.shape[1:]))
    for i in range(windows.shape[0]):
        windows[i, :] = x[i:(i+window_size), ]
    return windows

sliding_window_view(tf.constant(pwm_pad), 5)

In [ ]:
sliding_window(pwm_pad, 5)

In [ ]:
(1, 1, *pwm_pad.shape[1:])

In [ ]:
np.lib.stride_tricks.sliding_window_view(tf.constant(pwm), window_shape=(2, 4), axis=None).shape

In [ ]:
tf.tf_function()
def sliding_window_view(ndarray, **kwargs):
    tf.numpy_function(np.lib.stride_tricks.sliding_window_view(ndarray, **kwargs))


In [ ]:
tf.stack([pwm_tiled, pwm_tiled], axis=1)

In [ ]:
def sim(pmw_1, pmw_2):
    mask = tf.cast(tf.logical_and(tf.cast(tf.reduce_sum(pmw_1, axis=1), tf.bool), tf.cast(tf.reduce_sum(pmw_2, axis=1), tf.bool)), tf.float32)
    sim = tf.reduce_sum(tf.multiply(pmw_1, pmw_2), axis=1)
    sim_masked = tf.multiply(sim, mask)
    return tf.reduce_sum(sim_masked)

In [ ]:
tf.map_fn(apply_sim, c)

In [ ]:
%%timeit
tf.map_fn(apply_sim, c)

In [ ]:
mask.numpy()[::-1]

In [ ]:
def motif_point_similarity(pmw_1, pmw_2):
    mask = tf.cast(tf.logical_and(tf.cast(tf.reduce_sum(pmw_1, axis=1), tf.bool), tf.cast(tf.reduce_sum(pmw_2, axis=1), tf.bool)), tf.float32)
    sim = tf.reduce_sum(tf.multiply(pmw_1, pmw_2), axis=1)
    sim_masked = tf.multiply(sim, mask)
    return tf.reduce_sum(sim_masked)

motif_point_similarity(pwm, pwm)

In [ ]:
%%timeit
motif_point_similarity(pwm, pwm)

In [ ]:
a = kmers_onehot[0]
b = kmers_onehot[9]
print(a)
print(b)

In [ ]:
a

In [ ]:
class DistConv(tf.keras.layers.Conv1D):
    def __init__(self):
        super(DistConv, self).__init__(self)

dc = DistConv()
dc(a, b)

In [ ]:
def dist(a, b):
    padding = int(min([a.shape[0], b.shape[0]])/2)
    a_pad = tf.pad(a, [[padding, padding], [0, 0]])

    dists = []
    for i in range(a_pad.shape[0] - b.shape[0] + 1):
        a_pad_loc = a_pad[i:(i+5)]
        dists.append(tf.reduce_sum(a_pad_loc * b))
    return max(dists).numpy()

dist(a, b)

In [ ]:
a_pad = tf.pad(a, [[2, 2], [0, 0]])
a_pad

In [ ]:
for i in range(a_pad.shape[0] - b.shape[0] + 1):
    a_pad_loc = a_pad[i:(i+5)]
    print(tf.reduce_sum(a_pad_loc * b))

a_pad_after = a_pad.numpy()
a_pad_after[1:(1 + 5)] = a_pad_after[1:(1 + 5)] + b
a_pad_after = a_pad_after / 2
a_pad_after

In [ ]:
class Alignment():
    def __init__(self, a, b=None):
        self.pwm = None
        self.support = None

In [ ]:
tf.reduce_sum(a*b)

In [ ]:
tf.keras.losses.KLD(a[0], b[1])

In [ ]:
np.convolve(a, b, mode='same')

In [ ]:
from scipy.signal import convolve2d
convolve2d(a, b, mode='same')